In [1]:
import os,sys,inspect
import json
import torch
currentdir = os.path.dirname(os.getcwd())
print(currentdir)
# parent_dir = os.path.abspath(os.path.join(currentdir, os.pardir))
root = os.path.dirname(currentdir)
print("root Folder Absoloute path: ", root)


import sys
sys.path.append(root)

import logging

logging.basicConfig(level=logging.INFO)

from IPython.display import Image

import __main__

__main__.__file__="main.py"

/home/hfaghihi/Framework/new/DomiKnowS/examples
root Folder Absoloute path:  /home/hfaghihi/Framework/new/DomiKnowS


In [14]:
from regr.data.reader import RegrReader
import torch


class SudokuReader(RegrReader):
    def getidval(self, item):
        return [1]
    def getsizeval(self, item):
        return 5, 5
    

In [18]:
from regr.graph import Graph, Concept, Relation
from regr.graph.logicalConstrain import orL, andL, existsL, notL, atLeastL, atMostL, ifL, nandL, V, exactL
from regr.graph import EnumConcept


Graph.clear()
Concept.clear()
Relation.clear()

with Graph('global') as graph:
    sudoku = Concept("sodoku")
    
    empty_entries = Concept("empty_entries")
    fixed_entries = Concept("fixed_entries")
    
    (sudoku_empty, sudoku_fixed) = sudoku.has_a(empty_entries, fixed_entries)
    
    empty_entry = Concept('empty_entry')
    (empty_rel, ) = empty_entries.contains(empty_entry)
    
    fixed_entry = Concept('fixed_entry')
    (fixed_rel, ) = fixed_entries.contains(fixed_entry)
    
    same_row = Concept("same_row")
    same_row_mixed = Concept("same_row_mixed")
    (same_row_arg1, same_row_arg2) = same_row.has_a(arg1=empty_entry, arg2=empty_entry)
    (same_row_mixed_arg1, same_row_mixed_arg2) = same_row_mixed.has_a(arg1=empty_entry, arg2=fixed_entry)
    
    same_col = Concept("same_col")
    same_col_mixed = Concept("same_col_mixed")
    (same_col_arg1, same_col_arg2) = same_col.has_a(arg1=empty_entry, arg2=empty_entry)
    (same_col_mixed_arg1, same_col_mixed_arg2) = same_col_mixed.has_a(arg1=empty_entry, arg2=fixed_entry)
    
    empty_entry_label = empty_entry(name="empty_entry_label", ConceptClass=EnumConcept, values=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])


    ### Constriants
    
    for val in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]:
        ### No same number in the same row between empty entries and empty entries and the fixed ones.
        ifL(empty_entry_label.val('x'), notL(orL(
            existsL(
                andL(
                    same_row('z', path=("x", same_row_arg1.reverse)), empty_entry_label.val('y', path=("z", same_row_arg2))
                )), 
            existL(
                andL(
                    same_row_mixed('z', path=("x", same_row_mixed_arg1.reverse)), empty_entry_label.val('y', path=("z", same_row_mixed_arg2))
                ))
        )))
        
        ### No same number in the same column between empty entries and empty entries and the fixed ones.
        ifL(empty_entry_label.val('x'), notL(orL(
            existsL(
                andL(
                    same_col('z', path=("x", same_col_arg1.reverse)), empty_entry_label.val('y', path=("z", same_col_arg2))
                )), 
            existL(
                andL(
                    same_col_mixed('z', path=("x", same_col_mixed_arg1.reverse)), empty_entry_label.val('y', path=("z", same_col_mixed_arg2))
                ))
        )))


main.py:25: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'step'), ('arg2', 'step')]) is used.
  before_arg1,


In [9]:
from regr.sensor.pytorch.sensors import FunctionalSensor, JointSensor, ReaderSensor, FunctionalReaderSensor
from regr.sensor.pytorch.learners import ModuleLearner
from regr.sensor.pytorch.relation_sensors import CompositionCandidateSensor
from regr.sensor.pytorch.query_sensor import DataNodeReaderSensor

class JointFunctionalReaderSensor(JointSensor, FunctionalReaderSensor):
    pass


sudoku['id'] = ReaderSensor(keyword='id')
empty_entries['indexes'] = ReaderSensor(keyword='empty_indexes')
fixed_entries['indexes'] = ReaderSensor(keyword='fixed_indexes')


def make_empty_indexes(*prev):
    pass

empty_entry[empty_rel, 'row', 'column'] = FunctionalSensor(empty_entries['indexes'], forward=make_empty_indexes)

def read_fix_values(*prev):
    pass

fixed_entries[fixed_rel, 'row', 'column', 'value'] = JointFunctionalReaderSensor(context['text'], keyword='fixed_values', forward=read_fix_values)


